In [155]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
import time

In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: h5py>=2.9.0 in c:\users\hrida\anaconda3\lib\site-packages (from tensorflow) (3.2.1)



You should consider upgrading via the 'C:\Users\hrida\anaconda3\python.exe -m pip install --upgrade pip' command.


In [215]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [216]:
from transformers import AutoTokenizer, AutoModel
import torch
ctokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
cmodel = AutoModel.from_pretrained("microsoft/codebert-base")

In [217]:
def get_code_embeddings(code_snippet, tokenizer=ctokenizer, model=cmodel):
    
    """
    
    Returns a vector of shape (1,768) resembling the embedding of the code provided
    
    Input : Code in Text
    Return : Tensor
    
    """
    code_tokens=tokenizer.tokenize(code_snippet)
    tokens=[tokenizer.cls_token]+[tokenizer.sep_token]+code_tokens+[tokenizer.eos_token]
    tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
    tokens_ids = tokens_ids[:500]
    context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]
    code_embedding = context_embeddings[:, 0, :]
    return code_embedding

In [218]:
def get_nl_embeddings(text, model_nl=sbert_model):
    
    """
    
    Returns a vector of size (1,768) that corresponds to the embedding of the text
    
    
    """
    
    return torch.from_numpy(model_nl.encode(text).reshape(1,-1))

In [219]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [220]:
a=get_code_embeddings("def func(): return 6")

In [221]:
q1 = get_nl_embeddings("Hello I am a boy")
#q1=q1.reshape(1,-1)

In [222]:
import torch 
import torch.nn as nn

In [223]:
class LinearNetwork(nn.Module):
    def __init__(self):
        super(LinearNetwork, self).__init__()
        self.input_layer = nn.Linear(768,1024)
        self.hidden_layers = nn.ModuleList([
            nn.Linear(1024,1024) for _ in range(7)
        ])
        self.output_layer = nn.Linear(1024, 768)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.input_layer(x)
        for hidden_layer in self.hidden_layers:
            x = torch.relu(hidden_layer(x))
        x = self.tanh(self.output_layer(x))
        return x

model = LinearNetwork()

print("The Model Architecture is", str(model))

The Model Architecture is LinearNetwork(
  (input_layer): Linear(in_features=768, out_features=1024, bias=True)
  (hidden_layers): ModuleList(
    (0-6): 7 x Linear(in_features=1024, out_features=1024, bias=True)
  )
  (output_layer): Linear(in_features=1024, out_features=768, bias=True)
  (tanh): Tanh()
)


In [224]:
class SemanticLoss(nn.Module):
    def __init__(self):
        super(SemanticLoss, self).__init__()

    def forward(self, inputs, targets):
        cos = nn.CosineSimilarity()
        loss = 1 - cos(inputs,targets)
        return loss.mean()

In [225]:
def make_suitable(text):
    text=text.replace("\n", ",")
    text=text.replace(".", ",")

In [58]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [60]:
optimizer.zero_grad()

In [55]:
op=model(q1)

In [ ]:
from sklearn.pair import cosin

In [64]:
loss = SemanticLoss()

In [65]:
l=loss(op,a)

In [68]:
l.backward()
optimizer.step()

In [83]:
import pandas as pd    
jsonObj = pd.read_json(path_or_buf='python_train_0.jsonl', lines=True)

In [115]:
code=jsonObj.code_tokens
string=jsonObj.docstring_tokens

In [121]:
tokens=code[0]

In [161]:
code[1]

'def predict ( X_img_path  knn_clf = None  model_path = None  distance_threshold = 06 ) : if not os  path  isfile ( X_img_path ) or os  path  splitext ( X_img_path ) [ 1 ] [ 1 : ] not in ALLOWED_EXTENSIONS : raise Exception ( "Invalid image path: {}"  format ( X_img_path ) ) if knn_clf is None and model_path is None : raise Exception ( "Must supply knn classifier either thourgh knn_clf or model_path" ) # Load a trained KNN model (if one was passed in) if knn_clf is None : with open ( model_path  \'rb\' ) as f : knn_clf = pickle  load ( f ) # Load image file and find face locations X_img = face_recognition  load_image_file ( X_img_path ) X_face_locations = face_recognition  face_locations ( X_img ) # If no faces are found in the image return an empty result if len ( X_face_locations ) == 0 : return [ ] # Find encodings for faces in the test iamge faces_encodings = face_recognition  face_encodings ( X_img  known_face_locations = X_face_locations ) # Use the KNN model to find the best mat

In [96]:
code_tokens=tokenizer.tokenize(code[0])

In [226]:
def make_sentence(doc_tokens):
    text=""
    for token in doc_tokens:
        text +=token + " "
    text=text.replace(".", "")
    text=text.replace("," , "")
    return text

In [227]:
string=string.apply(make_sentence)
code=code.apply(make_sentence)

In [229]:
start = time.time()
string_set1.apply(get_nl_embeddings)
end=time.time()
print(f"The elapsed time was {end-start} seconds.")

The elapsed time was 32.7270290851593 seconds.


In [228]:
string_set1= string[:100]
code_set1 = code[:100]

In [ ]:
start = time.time()

end=time.time()
print(f"The elapsed time was {end-start} seconds.")